#DataDownload

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
#!cp /content/drive/MyDrive/kagglesjsu.json ~/.kaggle/
!mv kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c odsc-2024-nvidia-hackathon

100% 3.71G/3.72G [00:39<00:00, 192MB/s]
100% 3.72G/3.72G [00:39<00:00, 101MB/s]


In [ ]:
!unzip -q odsc-2024-nvidia-hackathon.zip

In [ ]:
!nvidia-smi

Sun Nov  3 00:19:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install cudf-cu12 cuml-cu12 --extra-index-url=https://pypi.ngc.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproje

In [ ]:
!pip install xgboost

#Data Load

In [ ]:
import cudf
import pandas as pd
import numpy as np
import cupy as cp
import cuml
from cuml.model_selection import train_test_split
from cuml.preprocessing import Normalizer
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from cuml.decomposition import PCA
from cuml.metrics.regression import mean_squared_error
from cuml import ForestInference

In [ ]:
df = cudf.read_csv("train.csv", nrows=1000, index_col="id")
#df = df.drop(['trickortreat', 'kingofhalloween'], axis=1)
# view a random sample of 10 rows:
#df.sample(10)

In [ ]:
feature_cols = [col for col in df.columns if col != 'y']
target_col = 'y'

In [ ]:
Y = df[target_col]
df = df.drop(target_col, axis=1)

In [ ]:
df.shape, Y.shape

((1000, 106), (1000,))

#Data Preprocessing

In [ ]:
# Apply log transformation to skewed features
# for col in df.columns:
#     cap = df[col].quantile(0.99)
#     df[col] = df[col].clip(upper=cap)
# print("Outliers capped at the 99th percentile for each feature.")

In [ ]:
int_columns = df.select_dtypes(include=['int64']).columns
float_columns = df.select_dtypes(include=['float64']).columns
for col in int_columns:
    df[col] = df[col].replace(-1, None)
int_means = df[int_columns].mean().astype('int64')
float_means = df[float_columns].mean()


df[int_columns] = df[int_columns].fillna(int_means)
df[float_columns] = df[float_columns].fillna(float_means)
# import numpy as np
# # Replace -1 with NaN, then calculate means
# means = df.replace(-1, None).mean()

# # Replace both NaN and -1 values with the calculated means
# df = df.replace(-1, None).fillna(means)

In [ ]:
for column in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        median = df[column].median()

        # Create a boolean mask for outliers
        outlier_mask = (df[column] < lower_bound) | (df[column] > upper_bound)

        # Replace outliers with median, maintaining the original dtype
        if df[column].dtype == 'int64':
            df[column] = df[column].astype('float64')
            df[column] = df[column].where(~outlier_mask, median)
            df[column] = df[column].astype('int64')
        else:
            df[column] = df[column].where(~outlier_mask, median)

In [ ]:
n_samples = len(df)
n_test = int(0.33 * n_samples)

# Create a series of random numbers
random_series = cudf.Series(cp.random.rand(n_samples))

# Add the random series as a column to X
df['random'] = random_series

# Sort X by the random column
X_sorted = df.sort_values('random').reset_index(drop=True)

# Remove the random column
X_sorted = X_sorted.drop('random', axis=1)

# Sort y using the same order as X
y_sorted = Y.iloc[df.sort_values('random').index].reset_index(drop=True)

# Split the sorted dataframes
X_train = X_sorted.iloc[n_test:]
X_test = X_sorted.iloc[:n_test]
y_train = y_sorted.iloc[n_test:]
y_test = y_sorted.iloc[:n_test]

In [ ]:
del df
#X_train, X_test, y_train, y_test = train_test_split(df, Y, shuffle=False, test_size=0.33, random_state=42)

# Print the shapes to verify the split
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)
print("Training label shape:", y_train.shape)
print("Testing label shape:", y_test.shape)

Training set shape: (7370000, 104)
Testing set shape: (3630000, 104)
Training label shape: (7370000,)
Testing label shape: (3630000,)


In [ ]:
# scaler = Normalizer().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

# from cuml.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


from cuml.preprocessing import RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
pca_float = PCA(n_components = 96)
pca_float.fit(X_train)
X_train = pca_float.transform(X_train)
X_test = pca_float.transform(X_test)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
params = {
    'objective': 'reg:squarederror',  # This is for regression tasks
    'eval_metric': 'rmse',  # Use RMSE as the evaluation metric
    'device':'cuda', # Specify which GPU to use
    'max_depth':4,
    'learning_rate':0.03,
    'tree_method':'gpu_hist',
    'subsample':0.8,
    'colsample_bytree':0.8,
    'predictor':'gpu_predictor',
    'reg_alpha':2,
    'reg_lambda':10,
    'gamma':0.1,
    'min_child_weight':8
    }

# 'reg_alpha':0.1,
#     'reg_lambda':0.1

In [ ]:
num_rounds = 10000  # Number of boosting rounds
model = xgb.train(
    params,
    dtrain,
    num_rounds,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=40,
    verbose_eval=10
)

best_model = model.copy()
best_model.best_iteration = model.best_iteration

[0]	train-rmse:666.16207	test-rmse:665.91985


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:33:41] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:33:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators", "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


[10]	train-rmse:658.27520	test-rmse:658.00207
[20]	train-rmse:654.20238	test-rmse:653.90942
[30]	train-rmse:651.36584	test-rmse:651.05720
[40]	train-rmse:649.65873	test-rmse:649.34171
[50]	train-rmse:648.65536	test-rmse:648.33905
[60]	train-rmse:647.93421	test-rmse:647.61848
[70]	train-rmse:647.41346	test-rmse:647.10179
[80]	train-rmse:647.04391	test-rmse:646.73798
[90]	train-rmse:646.72808	test-rmse:646.43182
[100]	train-rmse:646.46817	test-rmse:646.18292
[110]	train-rmse:646.23842	test-rmse:645.96299
[120]	train-rmse:646.03582	test-rmse:645.76985
[130]	train-rmse:645.85778	test-rmse:645.59998
[140]	train-rmse:645.70138	test-rmse:645.45325
[150]	train-rmse:645.55321	test-rmse:645.31096
[160]	train-rmse:645.42618	test-rmse:645.19016
[170]	train-rmse:645.28665	test-rmse:645.05788
[180]	train-rmse:645.18104	test-rmse:644.95746
[190]	train-rmse:645.08580	test-rmse:644.86891
[200]	train-rmse:644.98720	test-rmse:644.77594
[210]	train-rmse:644.89715	test-rmse:644.69155
[220]	train-rmse:644.7

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:34:45] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
preds = best_model.predict(dtest)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:28:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
preds

array([42321.73 , 42345.492, 42075.477, ..., 42376.44 , 42301.473,
       42487.145], dtype=float32)

In [ ]:
df_test = cudf.read_csv("test.csv", index_col="id")
df_test = df_test.drop(['trickortreat', 'kingofhalloween','y'], axis=1)

In [ ]:
int_columns = df_test.select_dtypes(include=['int64']).columns
for col in int_columns:
    df_test[col] = df_test[col].replace(-1, None)
float_columns = df_test.select_dtypes(include=['float64']).columns
int_means = df_test[int_columns].mean().astype('int64')
float_means = df_test[float_columns].mean()


df_test[int_columns] = df_test[int_columns].fillna(int_means)
df_test[float_columns] = df_test[float_columns].fillna(float_means)

In [ ]:
X = df_test[feature_cols]

In [ ]:
for column in X.select_dtypes(include=['float64', 'int64']).columns:
  Q1 = X[column].quantile(0.25)
  Q3 = X[column].quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  median = X[column].median()

  # Create a boolean mask for outliers
  outlier_mask = (X[column] < lower_bound) | (X[column] > upper_bound)

  # Replace outliers with median, maintaining the original dtype
  if X[column].dtype == 'int64':
    X[column] = X[column].astype('float64')
    X[column] = X[column].where(~outlier_mask, median)
    X[column] = X[column].astype('int64')
  else:
    X[column] = X[column].where(~outlier_mask, median)

In [ ]:
X = scaler.transform(X)

In [ ]:
X = pca_float.transform(X)

In [ ]:
dfinaltest = xgb.DMatrix(X)

In [ ]:
final_preds = best_model.predict(dfinaltest)
final_preds

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:35:01] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


array([42213.305, 42290.582, 42452.062, ..., 42166.74 , 42293.766,
       42372.473], dtype=float32)

In [ ]:
submission_df = cudf.DataFrame({'id': df_test.index, 'y': final_preds})

In [ ]:
submission_df.to_csv("submission.csv", index=False)

In [ ]:
model.save_model("nvidiamodel.model")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:24:26] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [ ]:
rmse = mean_squared_error(y_test, preds, sample_weight=None, multioutput='uniform_average', squared=False)

In [ ]:
rmse

array(648.39581652)

In [ ]:
model_path = 'xgb.model'
num_rounds = 1000
max_depth = 20

In [ ]:
def train_xgboost_model(dtrain, dtest, model_path='xgb.model',num_rounds=100, early_stopping_rounds=10):

    # set the xgboost model parameters
    params = {
        'verbosity': 0,
        'eval_metric':'rmse',
        'objective':'reg:squarederror',
        'tree_method': 'gpu_hist',
        'device':'cuda'
    }


    # train the xgboost model
    trained_model = xgb.train(params, dtrain, num_rounds,evals=[(dtrain, 'train'), (dtest, 'test')],early_stopping_rounds=early_stopping_rounds,
    verbose_eval=10)

    # save the trained xgboost model
    trained_model.save_model(model_path)

    return trained_model

In [ ]:
def predict_xgboost_model(dtest, xgb_model):

    # predict using the xgboost model
    #dvalidation = xgb.DMatrix(X_validation, label=y_validation)
    predictions = xgb_model.predict(dtest)

    # convert the predicted values from xgboost into class labels
    predictions = cp.around(predictions)

    return predictions

In [ ]:
%%time
# train the xgboost model
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
xgboost_model = train_xgboost_model(
    dtrain,
    dtest,
    model_path,
    num_rounds,
    10)

[0]	train-rmse:658.82326	test-rmse:657.52022
[10]	train-rmse:646.76784	test-rmse:646.79239
[20]	train-rmse:645.41956	test-rmse:646.36479
[30]	train-rmse:644.35828	test-rmse:646.27164
[40]	train-rmse:643.13927	test-rmse:646.30495
[42]	train-rmse:642.99571	test-rmse:646.30650
CPU times: user 2.55 s, sys: 165 ms, total: 2.72 s
Wall time: 4.94 s


In [ ]:
%%time
# test the xgboost model
trained_model_preds = predict_xgboost_model(
    dtest,
    xgboost_model
)

CPU times: user 15.7 ms, sys: 1.35 ms, total: 17.1 ms
Wall time: 11.5 ms


In [ ]:
rmse = mean_squared_error(y_test, trained_model_preds, sample_weight=None, multioutput='uniform_average', squared=False)
rmse

array(648.02944297)

#Training on Full Data

In [ ]:
import cudf
import cupy as cp
import xgboost as xgb
from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# Assuming X and Y are cuDF DataFrames
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# Standardize features
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

# Combine all data
X_all = cudf.concat([X_train, X_test], axis=0)
y_all = cudf.concat([y_train, y_test], axis=0)

# Convert to CuPy arrays for easier indexing
X_all = X_all.to_cupy()
y_all = y_all.to_cupy()

# Set up parameters
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'device': 'cuda'
}

# Perform cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_index, val_index in kf.split(X_all):
    X_train_cv, X_val_cv = X_all[train_index], X_all[val_index]
    y_train_cv, y_val_cv = y_all[train_index], y_all[val_index]

    dtrain_cv = xgb.DMatrix(X_train_cv, label=y_train_cv)
    dval_cv = xgb.DMatrix(X_val_cv, label=y_val_cv)

    model_cv = train_xgboost_model(
    dtrain_cv,
    dval_cv,
    model_path,
    num_rounds,
    10)

    cv_scores.append(model_cv.best_score)

# Calculate mean CV score using numpy
print(f"Mean CV score: {np.mean(cv_scores)}")

[0]	train-rmse:658.06667	test-rmse:659.17820
[10]	train-rmse:646.92765	test-rmse:648.39225
[20]	train-rmse:646.08067	test-rmse:647.78510
[30]	train-rmse:645.65973	test-rmse:647.56428
[40]	train-rmse:645.19823	test-rmse:647.39739
[50]	train-rmse:644.89680	test-rmse:647.31840
[60]	train-rmse:644.63219	test-rmse:647.27564
[70]	train-rmse:644.35469	test-rmse:647.23046
[80]	train-rmse:644.06566	test-rmse:647.21894
[90]	train-rmse:643.81988	test-rmse:647.21943


XGBoostError: [12:15:11] /workspace/src/c_api/../common/device_helpers.cuh:393: Memory allocation error on worker 0: std::bad_alloc: cudaErrorMemoryAllocation: out of memory
- Free memory: 2303524864
- Requested memory: 4505600000

Stack trace:
  [bt] (0) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x22dcbc) [0x79d4e9a2dcbc]
  [bt] (1) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x8a9966) [0x79d4ea0a9966]
  [bt] (2) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xec63c) [0x79d4e98ec63c]
  [bt] (3) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x924b2d) [0x79d4ea124b2d]
  [bt] (4) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x9262f7) [0x79d4ea1262f7]
  [bt] (5) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xa292b3) [0x79d4ea2292b3]
  [bt] (6) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xa2a092) [0x79d4ea22a092]
  [bt] (7) /usr/local

In [ ]:
X_all = cudf.concat([X_train, X_test], axis=0)
y_all = cudf.concat([y_train, y_test], axis=0)

# Convert to CuPy arrays for easier indexing
X_all = X_all.to_cupy()
y_all = y_all.to_cupy()

In [ ]:
del train_xgboost_model

In [ ]:
# Train final model on all data
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'device': 'cuda'
}
dall = xgb.DMatrix(X_all, label=y_all)
num_rounds = 90  # You might want to adjust this based on CV results
model_final = xgb.train(params, dall, num_rounds)

#PredictionsonTestData

In [ ]:
#del df
del X_train
del X_test
del y_train
del y_test
del X_all
del y_all

NameError: name 'df' is not defined

In [ ]:
df_test = cudf.read_csv("test.csv", index_col="id")
df_test = df_test.drop(['trickortreat', 'kingofhalloween','y'], axis=1)
# view a random sample of 10 rows:
#df_test.sample(10)

In [ ]:
int_columns = df_test.select_dtypes(include=['int64']).columns
for col in int_columns:
    df_test[col] = df_test[col].replace(-1, None)
float_columns = df_test.select_dtypes(include=['float64']).columns
int_means = df_test[int_columns].mean().astype('int64')
float_means = df_test[float_columns].mean()


df_test[int_columns] = df_test[int_columns].fillna(int_means)
df_test[float_columns] = df_test[float_columns].fillna(float_means)

In [ ]:
X = df_test[feature_cols]

In [ ]:
for column in X.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = X[column].quantile(0.25)
        Q3 = X[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        median = X[column].median()

        # Create a boolean mask for outliers
        outlier_mask = (X[column] < lower_bound) | (X[column] > upper_bound)

        # Replace outliers with median, maintaining the original dtype
        if X[column].dtype == 'int64':
            X[column] = X[column].astype('float64')
            X[column] = X[column].where(~outlier_mask, median)
            X[column] = X[column].astype('int64')
        else:
            X[column] = X[column].where(~outlier_mask, median)

In [ ]:
X = scaler.transform(X)

In [ ]:
X = pca_float.transform(X)

In [ ]:
dfinaltest = xgb.DMatrix(X)

In [ ]:
final_model_preds = predict_xgboost_model(
    dfinaltest,
    model
)

In [ ]:
submission_df = cudf.DataFrame({'id': df_test.index, 'y': final_model_preds})

In [ ]:
len(final_model_preds)

1000000

In [ ]:
len(df_test)

1000000

In [ ]:
submission_df.head()

,id,y
0,12000000,42119.0
1,12000001,42348.0
2,12000002,42468.0
3,12000003,42187.0
4,12000004,42460.0


In [ ]:
submission_df.to_csv("submission.csv", index=False)

#SVD

In [ ]:
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

In [ ]:
n_samples = 11000000 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 96

random_state = 42



In [ ]:
ols_cuml = cuLinearRegression(fit_intercept=True,
                              algorithm='eig')

ols_cuml.fit(X_train, y_train)

LinearRegression()

In [ ]:
predict_cuml = ols_cuml.predict(X_test)

In [ ]:
predict_cuml

0          154.792058
1          -92.427271
2          170.879904
3         -101.685296
4         -210.103041
              ...    
3629995    188.546089
3629996    -77.905501
3629997    117.191685
3629998   -192.827683
3629999     17.790224
Length: 3630000, dtype: float64

In [ ]:
from cuml.linear_model import MBSGDRegressor as cumlMBSGDRegressor

In [ ]:
cu_mbsgd_regressor = cumlMBSGDRegressor(learning_rate='adaptive',
                                        eta0=0.1, epochs=2000,
                                        fit_intercept=False,
                                        batch_size=128, tol=50.0,
                                        penalty='elasticnet',
                                        loss='squared_loss',
                                        l1_ratio=0.5,verbose=20,shuffle=True,
                                        alpha=0.5)

In [ ]:
cu_mbsgd_regressor.fit(X_train, y_train)

#DUMMY

In [ ]:
mean_dict={}
negative_count_dict={}
for col in df.columns:
  if type(df[col]) == int:
    mean_dict[col] = df[col].mean()



In [ ]:
count_negative_one = (df['celebrate'] == -1).sum()
count_negative_one

0

In [ ]:
less10 = (df['celebrate'] < 50).sum()
less10

1198

In [ ]:
df['celebrate'].mean()

19554.713143

In [ ]:
len(df)

1000000

In [ ]:
cnts = df['trickortreat'].value_counts()
cnts

KeyError: 'trickortreat'

In [ ]:
len(df['kingofhalloween'].value_counts())

In [ ]:
len(cnts)

In [ ]:
import cudf
import cuml
from cuml.decomposition import PCA

In [ ]:
import cudf
print(cudf.__version__)

In [ ]:
!conda install -c rapidsai -c conda-forge -c nvidia rapids=24.10 python=3.10 cuda-version=12.2 xgboost

In [ ]:
!python --version

In [ ]:
!nvidia-smi

In [ ]:
!pip install cudf-cu12 cuml-cu12 --extra-index-url=https://pypi.ngc.nvidia.com

In [ ]:
import cupy as cp

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c odsc-2024-nvidia-hackathon

In [ ]:
!unzip -q odsc-2024-nvidia-hackathon.zip

In [ ]:
import cudf

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("train.csv", nrows=1000000, index_col="id")
df = df.drop(['trickortreat', 'kingofhalloween'], axis=1)
# view a random sample of 10 rows:
df.sample(10)

#Text Processing

In [ ]:
import re
filters = ['!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '/', ':', ';', '<', '=', '>',
           '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '~', "'", ',', '—']

text_col_sample = df[['trickortreat']].head(5)
text_col_sample['trickortreat'].to_pandas()

pattern = '|'.join(map(re.escape, filters))

text_col_sample['trickortreat_clean'] = text_col_sample['trickortreat'].str.replace(pattern, ' ', regex=True)
text_col_sample['trickortreat_clean'] = text_col_sample['trickortreat_clean'].str.lower()

# # Display the result
# print(df[['trickortreat', 'trickortreat_clean']])

# text_col_sample['trickortreat_clean'] = text_col_sample['trickortreat'].str.replace_multi(filters, ' ', regex=False)
# text_col_sample['trickortreat_clean'] = text_col_sample['trickortreat_clean'].str.lower()
text_col_sample['trickortreat_clean'].to_pandas()

,trickortreat_clean
id,
0,century totalrecall
1,payday consternation
2,jackpot tragedy
3,cape experiences
4,shadow issues


In [ ]:
import cudf
import cuml
from cuml.feature_extraction.text import HashingVectorizer, CountVectorizer, TfidfVectorizer

# Sample data
data = cudf.Series(['This is the first document.',
                    'This document is the second document.',
                    'And this is the third one.',
                    'Is this the first document?'])

# 1. Using HashingVectorizer
hash_vectorizer = HashingVectorizer(n_features=2**10)
hash_vectors = hash_vectorizer.fit_transform(data)

print("HashingVectorizer output:")
print(hash_vectors.shape)
print(hash_vectors.todense())

# 2. Using CountVectorizer
count_vectorizer = CountVectorizer()
count_vectors = count_vectorizer.fit_transform(data)

print("\nCountVectorizer output:")
print(count_vectors.shape)
print(count_vectors.todense())

# 3. Using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(data)

print("\nTfidfVectorizer output:")
print(tfidf_vectors.shape)
print(tfidf_vectors.todense())

# 4. Character n-grams with TfidfVectorizer
char_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
char_vectors = char_vectorizer.fit_transform(data)

print("\nCharacter n-gram TfidfVectorizer output:")
print(char_vectors.shape)
print(char_vectors.todense())

HashingVectorizer output:
(4, 1024)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

CountVectorizer output:
(4, 9)
[[0. 1. 1. 1. 0. 0. 1. 0. 1.]
 [0. 2. 0. 1. 0. 1. 1. 0. 1.]
 [1. 0. 0. 1. 1. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 1. 0. 1.]]

TfidfVectorizer output:
(4, 9)
[[0.         0.46979138 0.5802858  0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108868 0.         0.53864765
  0.28108868 0.         0.28108868]
 [0.5118485  0.         0.         0.2671038  0.5118485  0.
  0.2671038  0.5118485  0.2671038 ]
 [0.         0.46979138 0.5802858  0.38408524 0.         0.
  0.38408524 0.         0.38408524]]

Character n-gram TfidfVectorizer output:
(4, 82)
[[0.13160315 0.13160315 0.16255607 0.16255607 0.13160315 0.13160315
  0.         0.         0.         0.         0.10759419 0.10759419
  0.         0.         0.         0.         0.13160315 0.13160315
  0.         0.         0.     

ModuleNotFoundError: No module named 'nvstrings'

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for nvstrings
  Running setup.py clean for nvstrings
Failed to build nvstrings
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (nvstrings)
